# Importing packages and reading the table

In [1]:
import csv
import pandas as pd

In [2]:
df = pd.read_excel("messy_sample_sheet.xlsx")
df

,SID,Condition,Sex,Batch,PMI,RIN,Liver,BrainPH,BrainWeight,LeftRightBrain,Smoking
0,SRR15466722,Control,male,2,29,"4,9",MildSteatosis,6.12,1320,Left,Never
1,SRR15466723,Control,Female,1,15,7.3,Normal,6.93,1330,Right,NaN
2,SRR15466724,Control,female,2,11,3.4,Normal,6.21,1200,Left,Current
3,SRR15466725,AlcoholUseDisorder,female,1,38,7.8,Normal,6.54,1180,Right,Current
4,SRR15466742,Control,female,2,23,4.6,Congestion,6.17,1340,Left,NAN
5,SRR15466743,AlcoholUseDisorder,male,1,39.5,8,Cirrhosis,6.34,1412,Left,Current
6,SRR15466744,Control,femalle,1,29.5,8.2,Normal,6.78,1195,Left,Never
7,SRR15466745,AlcoholUseDisorder,female,2,37,"6,8",Steatosis,6.95,1139,Right,Never
8,SRR15466746,AlcoholUseDisorder,male,2,61,7.3,Steatosis,6.79,1670,Left,Current
9,SRR15466729,Control,male,1,12,7.6,Steatosis,6.39,1631,Left,Never


# Things to check/fix

- If SIDs are all unique
- Condition, Sex, Liver, LeftRightBrain, Smoking values standartized (no values meaning the same thing)
- Numbers with with correct demoninator (either comma or dot)
- Find missing values, decide what to do with them

## Initial step: removing sample that's obviously bad

In other words we can check which samples have a lot of missing values and it's easier to check with numeric values. Even though Batch is categorical, numbers are used.

In [3]:
numeric_cols = ["Batch", "PMI", "RIN", "BrainPH", "BrainWeight"]
sids_to_remove = []
for i in range(df.shape[0]):
    failed_check = 0
    for nc in numeric_cols:
        val = str(df[nc][i]).replace(",", ".")
        try:
            float(val)
        except ValueError:
            failed_check += 1
    if failed_check > 0:
        print(f"{df['SID'][i]} failed {failed_check} checks")
        sids_to_remove.append(df["SID"][i])
sids_to_remove

SRA15500000 failed 5 checks


['SRA15500000']

In [4]:
print(df.shape)
df = df.loc[~df["SID"].isin(sids_to_remove)]
print(df.shape)

(25, 11)
(24, 11)


## Checking if SIDs are all unique

- set() leaves only unique values
- True means that there are no duplicate values

In [5]:
len(df["SID"]) == len(set(df["SID"]))

True

## Standartizing Condition values

In [6]:
set(df["Condition"])

{'AlcoholUseDisorder', 'AlcoholUsedisorder', 'Control'}

In [7]:
conds = [str(val).replace('AlcoholUsedisorder', 'AlcoholUseDisorder') for val in df["Condition"]]
df = df.copy()
df.loc[:, "Condition"] = conds

In [8]:
set(df["Condition"])

{'AlcoholUseDisorder', 'Control'}

## Standartizing Sex values

Variation is bigger so syntax won't be so easy as with Condition

In [9]:
set(df["Sex"])

{'Female', 'Male', 'f', 'female', 'femalle', 'male'}

In [10]:
new_values = []
for s in df["Sex"]:
    if s in ['Female', 'f', 'female', 'femalle']:
        new_values.append("Female")
    elif s in ['Male', 'male']:
        new_values.append("Male")
    else:
        print(f"Undetected value: {s}")
df.loc[:, "Sex"] = new_values

In [11]:
set(df["Sex"])

{'Female', 'Male'}